In [31]:
def choose(n, k):
    if k == 0: #BASE CASE 1: choosing 0 items from n
        return 1 #return 1 for the only way to choose 0 items from n
    if k > n: #BASE CASE 2: choosing more items from n than n has
        return 0 #Return 0 for not being able to choose more items than available
    return n * choose(n - 1, k - 1) // k #Chooses first item from the set and multiplies the number of ways to choose k-1 items from the remaining n-1 items;
                                            #integer division by k to ensure we return an integer and to only count combinations by removing the arrangement factor

In [32]:
def generate_combinations(s, k):
    if k == 0:  #BASE CASE 1: k=0; selecting 0 items from 's'
        return [[]] #return nothing (no combinations)
    elif not s: #BASE CASE 2: s=0; selecting 'k' items from an empty list
        return [] #return nothing (no possibilities)
    else: #RECURSIVE CASE
        with_first = generate_combinations(s[1:], k-1) #Combinations w/ the first item; remove the first item from the list and find combinations with a decremented k by 1
        for combo in with_first: #Loop through found combinations w/ k-1 
            combo.append(s[0]) #Append the first element to the found combinations to make them size k
        without_first = generate_combinations(s[1:], k)#Combinations w/o the first item; remove the first item from the list and generate combinations w/ the rest of the elements
        return with_first + without_first #concatenate results w/ and w/o first element to get final output

In [34]:
#No Allelic Copies Probability; Equation 1
#Params: int matrix N, int g, int i, int j
def q_ijg(N, g, i, j):
    Nj = sum(allele[j] for allele in N)
    return choose(Nj - N[i][j], g) / choose(Nj, g)

In [35]:
#At Least One Copy Probability
#Params: float Qijg
def p_ijg(Qijg):
    return 1 - Qijg

In [36]:
#Allelic Richness; Equation 2
#Params: int matrix N, int g, int j
def alpha_jg(N, g, j):
    I = len(N)
    sum_value = 0
    for i in range(I):
        Qijg = q_ijg(N, g, i, j)
        sum_value += p_ijg(Qijg)
    return sum_value

In [37]:
#Private Allelic Richness; Equation 3
def pi_jg(N, g, j):
    I,J = len(N), len(N[0])
    sum_value = 0
    for i in range(I):
        Pijg = p_ijg(q_ijg(N, g, i, j))
        product = 1
        for j_prime in range(J):
            if j_prime != j:
                Qij_primeg = q_ijg(N, g, i, j_prime)
                product *= Qij_primeg
        sum_value += Pijg * product
    return sum_value

In [38]:
#Private Allelic Combinations Across Populations; Equation 4
def pi_mgk(N, g, k, m):
    I, J = len(N), len(N[0])
    all_pop_combinations = generate_combinations(list(range(J)), k)
    specific_combination = all_pop_combinations[m]
    sum_value = 0
    for i in range(I):
        product_P = 1
        for j in specific_combination:
            Pijg = p_ijg(q_ijg(N, g, i, j))
            product_P *= Pijg

        S_without_Ckm = set(range(J)) - set(specific_combination)
        product_Q = 1
        for j_prime in S_without_Ckm:
            Qij_primeg = q_ijg(N, g, i, j_prime)
            product_Q *= Qij_primeg

        sum_value += product_P * product_Q
    return sum_value

In [39]:
#Missing Allelic Richness; Equation 5
def mu_jg(N, g, j):
    I, J = len(N), len(N[0])
    sum_value = 0
    for i in range(I):
        Qijg = q_ijg(N, g, i, j)
        product = 1
        for j_prime in range(J):
            if j_prime != j:
                Pij_primeg = q_ijg(N, g, i, j_prime)
                product *= Pij_primeg
        sum_value += Qijg * product
    return sum_value


In [55]:
# Example:
N = [
    [3, 2, 1],
    [0, 3, 2],
    [2, 0, 3],
    [4, 3, 8]
]
g = 2
k = 2
j = 2
m = 2  # Just a random selection for the demonstration.

print(f"Alpha for j={j} and g={g}:", alpha_jg(N, g, j))
print(f"Pi for j={j} and g={g}:", pi_jg(N, g, j))
print(f"Pi for m={m}, g={g} and k={k}:", pi_mgk(N, g, k, m))
print(f"Mu for j={j} and g={g}:", mu_jg(N, g, j))

Alpha for j=2 and g=2: 1.6483516483516485
Pi for j=2 and g=2: 0.44362680969823837
Pi for m=2, g=2 and k=2: 0.35338173731030875
Mu for j=2 and g=2: 0.819270015698587
